In [1]:
import os
import cv2
import timeit
import numpy as np
import tensorflow as tf

c:\users\dick\2\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\dick\2\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\dick\2\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\dick\2\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [4]:
camera = cv2.VideoCapture(0)

# Loads label file, strips off carriage return
label_lines = [line.rstrip() for line
               in tf.io.gfile.GFile('./Model_TF1_13_1/7-classifier_1_13_1_labels.txt')]

def grabVideoFeed():
    grabbed, frame = camera.read()
    return frame if grabbed else None

def initialSetup():
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
    start_time = timeit.default_timer()

    # This takes 2-5 seconds to run
    # Unpersists graph from file
    with tf.compat.v1.gfile.FastGFile('./Model_TF1_13_1/7-classifier_1_13_1.pb', 'rb') as f:
        graph_def = tf.compat.v1.GraphDef()#tf.GraphDef()
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name='')

    #print ('Took {} seconds to unpersist the graph'.format(timeit.default_timer() - start_time))

initialSetup()

with tf.compat.v1.Session() as sess:
    start_time = timeit.default_timer()
    #print(sess.graph.get_operations())
    
    # Feed the image_data as input to the graph and get first prediction
    softmax_tensor = sess.graph.get_tensor_by_name('dense_4/Sigmoid:0')
    #softmax_tensor = sess.graph.get_tensor_by_name('dense_2/Sigmoid:0')

    while True:
        frame = grabVideoFeed()

        cv2.imshow('Main', frame)
        if frame is None:
            raise SystemError('Issue grabbing the frame')

        
        float_caster = tf.cast(frame, tf.float32)
        dims_expander = tf.expand_dims(float_caster, 0);
        resized = tf.image.resize(dims_expander, [64, 64], method=tf.image.ResizeMethod.BILINEAR)
        normalized = tf.divide(tf.subtract(resized, [0]), [255])
        sess = tf.compat.v1.Session()
        numpy_final = sess.run(normalized)
        start_time = timeit.default_timer()

        # This takes 2-5 seconds as well
        predictions = sess.run(softmax_tensor, {'conv2d_3_input:0': numpy_final})
        #predictions = sess.run(softmax_tensor, {'conv2d_1_input:0': numpy_final})

        start_time = timeit.default_timer()

        # Sort to show labels of first prediction in order of confidence
        top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]

        max=0
        nodo=0
        for node_id in top_k:
            score = predictions[0][node_id]
            if score>max:
                nodo=node_id
                max=score
        human_string = label_lines[nodo]
        score = predictions[0][nodo]
        print('%s (score = %.5f)' % (human_string, score))
        
        if cv2.waitKey(1) == 13: #13 is the Enter Key
            sess.close()
            break
camera.release()
cv2.destroyAllWindows()

Instructions for updating:
Use tf.gfile.GFile.
Otros (score = 0.79140)
Otros (score = 0.74734)
Otros (score = 0.73988)
Otros (score = 0.72347)
Otros (score = 0.74589)
Otros (score = 0.73767)
Otros (score = 0.80735)
Otros (score = 0.78095)
Otros (score = 0.80457)
Otros (score = 0.78274)
Otros (score = 0.76210)
Otros (score = 0.81314)
Otros (score = 0.79614)
Otros (score = 0.78926)
Otros (score = 0.78162)
Otros (score = 0.80717)
Otros (score = 0.77071)
Otros (score = 0.79205)
Otros (score = 0.80934)
Otros (score = 0.84738)
Otros (score = 0.85582)
Otros (score = 0.88944)
Otros (score = 0.86691)
Otros (score = 0.81833)
Otros (score = 0.79332)
Otros (score = 0.62081)
Otros (score = 0.62183)
Otros (score = 0.55552)
Otros (score = 0.50434)
Otros (score = 0.50587)
